In [23]:
import sys
import torch
import tqdm
import numpy as np
import random
import os
import json
sys.path.append('../../')
from models.cnn.search_cnn import  SearchCNN, SearchCNNController
from models.cnn_darts_hypernet.search_cnn_darts_hypernet import SearchCNNControllerWithHyperNet
from models.cnn.one_hot_cnn import OneHotSearchCNNController

from configobj import ConfigObj

In [17]:
import utils
# get data with meta info
input_size, input_channels, n_classes, train_data, valid_data = utils.get_data(
    'fashionmnist', '../../data/', cutout_length=0, validation=True)

test_loader = torch.utils.data.DataLoader(valid_data,
                                           batch_size=64,
                                           shuffle=False,
                                           num_workers=1,
                                           pin_memory=True)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [83]:
# пример загрузки модели из random-search
seed = 50 # seed нужно выставлять обязательно, он должен соответствовать сиду, из которого проиходило обучение
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
config = ConfigObj('../../configs/mini_fmnist_hyper_final/fmnist_random.cfg')
config['device'] = 'cuda' # если cuda нет, можно и на cpu загрузить
model = OneHotSearchCNNController(**config)
model.load_state_dict(torch.load('../../searchs/mini_fmnist_random/best_{}.pth.tar'.format(seed)))
model.cuda()
model.eval()

[[0, 0], [3, 5, 1], [6, 4, 6, 5], [6, 6, 5, 5, 2]] [[7, 2], [7, 7, 4], [6, 3, 4, 3], [2, 2, 6, 4, 3]]


OneHotSearchCNNController(
  (net): OneHotCNN(
    (stem): Sequential(
      (0): Conv2d(1, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (cells): ModuleList(
      (0): SearchCell(
        (preproc0): StdConv(
          (net): Sequential(
            (0): ReLU()
            (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
          )
        )
        (preproc1): StdConv(
          (net): Sequential(
            (0): ReLU()
            (1): Conv2d(48, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
          )
        )
        (dag): ModuleList(
          (0): ModuleList(
            (0): MixedOp(
              (_ops): ModuleList(
                (0):

In [84]:
def accuracy(model, loader):
    total = 0
    correct = 0
    for x,y in loader:
        out = torch.argmax(model(x.cuda()), 1)
        total += x.shape[0]
        correct += torch.eq(out.detach().cpu(), y).numpy().sum()
    return correct/total

accuracy(model, test_loader)

0.9274

In [86]:
def param_number(model):
    # Поскольку в нашем дообучении из каждой mixed option берется ровно одна операция,
    # проще просуммировать все операции, которые нами не используются, и вычесть их из общего числа параметров модели    
    blacklist_parameters = 0
    for c in model.net.cells:
        if c.reduction:
            alphas = model.weights_reduce
        else:
            alphas = model.weights_normal
        for node, alpha in zip(c.dag, alphas):
            for mixed_op, alpha_int in zip(node, alpha):                
                ops_to_blacklist = [mixed_op._ops[id] for id in range(len(mixed_op._ops)) if id != alpha_int]
                for op in ops_to_blacklist:
                    for param in op.parameters():
                        blacklist_parameters += np.prod(list(param.shape))
    # считаем количество параметров
    total_parameters = -blacklist_parameters
    for param_name, param in model.net.named_parameters():
        # в DARTS есть еще вспомогательный линейный слой (auxiliary head), который используется, чтобы модель лучше обучалась на
        # первых клетках. Вспомогательный слой используется только на обучении, его не учитывают в параметрах
        # (в нашем текущем эксперименте на маленьких моделях вспомогательного слоя нет, но оставлю эту проверку
        # на будущее)
        if 'aux' in param_name:
            continue
        total_parameters += np.prod(list(param.shape))

    return total_parameters
param_number(model)            

109178